In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def make_ddelay(div_ratio, n):
    rv = []
    lo = math.floor(div_ratio)
    hi = lo + 1
    if 0.5 <= div_ratio - lo and 0 < n:
        rv.append(hi)
        hi += 1
        n -= 1
    while 0 < n:
        rv.append(lo)
        lo -= 1
        n -= 1
        if n <= 0:
            break
        rv.append(hi)
        hi += 1
        n -= 1
    return rv

In [ ]:
class FDFir(object):
    def __init__(self, n_harm, div_ratio, ndof):
        ddelay = np.array(make_ddelay(div_ratio, 2*n_harm+1+ndof)[::-1])
        phase_delay = -2*np.pi*ddelay[None, :]/div_ratio*np.arange(0, n_harm+1)[:, None]
        sys = np.hstack(
         (np.vstack((np.cos(phase_delay), np.sin(phase_delay[1:]))),
          np.vstack((np.ones((n_harm+1, 1)), np.zeros((n_harm, 1))))))
        constraint = np.linalg.solve(sys[:,0:sys.shape[0]], sys[:,sys.shape[0]:])
        if 1 <= ndof:
            b = np.zeros_like(ddelay, dtype=np.float64)
            b[:constraint.shape[0]] -= constraint[:,-1]
            a = np.vstack((-constraint[:,:-1], np.eye(constraint.shape[1]-1)))
            dof = np.linalg.lstsq(a, b, rcond=-1)
            coeff = np.hstack(
             (np.dot(constraint, np.hstack((-dof[0], [1]))),
              dof[0]))
        else:
            coeff = constraint[:,0]
        self.coeff = coeff
        self.ddelay = ddelay

    def transf(self, f_norm):
        return (np.exp(-2j*np.pi*self.ddelay[None, :]*f_norm[:, None])*self.coeff[None, :]).sum(axis=1)

In [ ]:
div_ratio = 48000/440; div_ratio

In [ ]:
hmax = math.floor(div_ratio+0.5)-1
f_norm_test = np.arange(0, 0.5, 0.01)
for harm in range(1, hmax+1):
    for ndof in range(2*(hmax-harm), 0, -1):
        try:
            fdfir = FDFir(harm, div_ratio, ndof)
            transf = fdfir.transf(f_norm_test)
            print(harm, ndof, np.abs(transf).max())
        except Exception:
            print(harm, ndof)